# Handwriting Recognition With Deep Learning

This notebook walks through a method for recognizing handwriting through classification of the [notMNIST](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html) character dataset.  This dataset is designed to look like the classic [MNIST](http://yann.lecun.com/exdb/mnist/) dataset, while looking a little more like real data: it's a harder task, and the data is a lot less 'clean' than MNIST.

A quick and simple two-layer convolutional neural network with max pooling filters and RELU nonlinearily activation functions, followed by one fully-connected layer, is fit to the data. The model obtains a ~8% error rate on a test set after just minutes on a personal CPU.  The model architecture is a simplified variant of the classic [LeNet5](http://yann.lecun.com/exdb/lenet/).  Implementation uses the TensorFlow library.

In [ ]:
# LIBRARIES
from __future__ import print_function

import random
import numpy as np
import tensorflow as tf

from six.moves import cPickle as pickle
from six.moves import range

import handwriting_recognition_data as hrd


# PROPRIETARY FUNCTIONS
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

--------
## Data Import and Preprocessing

The data is hosted on a Google server, and is accessed through a download routine found in `handwriting_recognition_data.py`.  

### DOWNLOADING TARBALLS

The `maybe_download` function returns the downloads as tarballs, which are extracted with the `maybe_extract` function.  If you have already downloaded the tarballs, split the data and pickled, please proceed to section "RESTORING".

In [ ]:
num_classes = 10 # number of letters for classification
np.random.seed(133)
image_size = 28  # Pixel width and height.
pixel_depth = 255.0  # Number of levels per pixel.
url = 'http://commondatastorage.googleapis.com/books1000/' #dir of notmsint data

#Download the binary file if not present, and make sure it's the right size
#reports the progress of a download
train_filename = hrd.maybe_download('notMNIST_large.tar.gz', url, 247336696)
test_filename = hrd.maybe_download('notMNIST_small.tar.gz', url, 8458043)

#Extract
train_folders = hrd.maybe_extract(train_filename)
test_folders = hrd.maybe_extract(test_filename)

### SPLITTING THE DATA

Split into training, validation, and test sets.  Randomize the data.

In [ ]:
# SPLIT DATA
train_size = 200000
valid_size = 10000
test_size = 10000

valid_dataset, valid_labels, train_dataset, train_labels = hrd.merge_datasets(
  train_datasets, train_size, valid_size)
_, _, test_dataset, test_labels = hrd.merge_datasets(test_datasets, test_size)

#RANDOMIZE
train_dataset, train_labels = hrd.randomize(train_dataset, train_labels)
test_dataset, test_labels = hrd.randomize(test_dataset, test_labels)
valid_dataset, valid_labels = hrd.randomize(valid_dataset, valid_labels)

# VERIFY DATA IS GOOD
print('Training:', train_dataset.shape, train_labels.shape)
print('Validation:', valid_dataset.shape, valid_labels.shape)
print('Testing:', test_dataset.shape, test_labels.shape)

### PICKLING

Now we load the data in a more manageable format. We load each class (letter) into a separate dataset, store them on disk and curate them independently. Later we'll merge them into a single dataset of manageable size.

We'll convert the entire dataset into a 3D array (image index, x, y) of floating point values, normalized to have approximately zero mean and standard deviation ~0.5 to make training easier down the road. A few images might not be readable, we'll just skip them.

If you have already pickled the tarballs, please proceed to section "RESTORING".

In [ ]:
# PICKLE
train_datasets = hrd.maybe_pickle(train_folders, 45000)
test_datasets = hrd.maybe_pickle(test_folders, 1800)

# SAVE
pickle_file = 'notMNIST.pickle'

try:
  f = open(pickle_file, 'wb')
  save = {
    'train_dataset': train_dataset,
    'train_labels': train_labels,
    'valid_dataset': valid_dataset,
    'valid_labels': valid_labels,
    'test_dataset': test_dataset,
    'test_labels': test_labels,
    }
  pickle.dump(save, f, pickle.HIGHEST_PROTOCOL)
  f.close()
except Exception as e:
  print('Unable to save data to', pickle_file, ':', e)
  raise

statinfo = os.stat(pickle_file)
print('Compressed pickle size:', statinfo.st_size)

### RESTORING

Restore data in memory from existing pickle file.

In [ ]:
# RESTORE DATA FROM PICKLE FILES
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

### REFORMATTING into a TensorFlow-friendly shape:

We need to reformat the data into a TF-friendly shape with the following attributes:
- convolutions need the image data formatted as a cube (width by height by channels)
- labels must be "1-hot encodings"

In [ ]:
train_dataset, train_labels = hrd.tf_reformat(train_dataset, train_labels, 
                                              image_size = 28, 
                                              num_labels = 10,
                                              num_channels = 1)

valid_dataset, valid_labels = hrd.tf_reformat(valid_dataset, valid_labels,  
                                              image_size = 28, 
                                              num_labels = 10,
                                              num_channels = 1)

test_dataset, test_labels = hrd.tf_reformat(test_dataset, test_labels, 
                                            image_size = 28, 
                                            num_labels = 10,
                                            num_channels = 1)

print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

--------
## Building and Running a CNN in Tensorflow

We use the Tensorflow library to automatically handle our computational and memory resources.  This consists of a "building the graph" operation and a "running the graph" operation.

### Build the TF Graph:

We build a small network with two convolutional layers + max-pooling and non-linear activation layers, followed by one fully connected layer.

In [ ]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64
image_size = 28 
num_labels = 10
num_channels = 1

graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  layer1_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, num_channels, depth], stddev=0.1))
  layer1_biases = tf.Variable(tf.zeros([depth]))
  layer2_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth, depth], stddev=0.1))
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
  layer3_weights = tf.Variable(tf.truncated_normal(
      [image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
  layer4_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_labels], stddev=0.1))
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
  # Model.
  def model(data):
    conv = tf.nn.conv2d(data, layer1_weights, [1, 1, 1, 1], padding='SAME') #SAME is zero padding
    maxpool = tf.nn.max_pool(conv,[1,2,2,1],[1,2,2,1],padding='SAME')
    hidden = tf.nn.relu(maxpool + layer1_biases)
    conv = tf.nn.conv2d(hidden, layer2_weights, [1, 1, 1, 1], padding='SAME')
    maxpool = tf.nn.max_pool(conv,[1,2,2,1],[1,2,2,1],padding='SAME')
    hidden = tf.nn.relu(maxpool + layer2_biases)
    shape = hidden.get_shape().as_list()
    reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
    hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
    return tf.matmul(hidden, layer4_weights) + layer4_biases
  
  # Training computation.
  logits = model(tf_train_dataset)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))

### Run the TF Graph:

In [ ]:
num_steps = 1001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 100 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))